# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 15 2022 10:50AM,241963,10,PRF-33379,Bio-PRF,Released
1,Jul 15 2022 10:47AM,241947,15,8278211,"Carwin Pharmaceutical Associates, LLC",Released
2,Jul 15 2022 10:39AM,241962,12,U0969,"Uniderm USA, Inc",Released
3,Jul 15 2022 10:39AM,241962,12,U0968,"Uniderm USA, Inc",Released
4,Jul 15 2022 10:36AM,241961,10,Enova-10311,Emerginnova,Released
5,Jul 15 2022 10:30AM,241960,10,PRF-33386,Bio-PRF,Released
6,Jul 15 2022 10:30AM,241960,10,PRF-33390,Bio-PRF,Released
7,Jul 15 2022 10:30AM,241960,10,PRF-33326,Bio-PRF,Released
8,Jul 15 2022 10:30AM,241960,10,PRF-33330,Bio-PRF,Released
9,Jul 15 2022 10:30AM,241960,10,PRF-33333,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,241958,Released,1
26,241960,Released,8
27,241961,Released,1
28,241962,Released,2
29,241963,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241958,NaN,NaN,1.0
241960,NaN,NaN,8.0
241961,NaN,NaN,1.0
241962,NaN,NaN,2.0
241963,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241871,0.0,1.0,0.0
241883,2.0,0.0,1.0
241888,0.0,0.0,1.0
241891,0.0,0.0,1.0
241894,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241871,0,1,0
241883,2,0,1
241888,0,0,1
241891,0,0,1
241894,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241871,0,1,0
1,241883,2,0,1
2,241888,0,0,1
3,241891,0,0,1
4,241894,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241871,,1,
1,241883,2,,1
2,241888,,,1
3,241891,,,1
4,241894,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 15 2022 10:50AM,241963,10,Bio-PRF
1,Jul 15 2022 10:47AM,241947,15,"Carwin Pharmaceutical Associates, LLC"
2,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc"
4,Jul 15 2022 10:36AM,241961,10,Emerginnova
5,Jul 15 2022 10:30AM,241960,10,Bio-PRF
13,Jul 15 2022 10:18AM,241958,10,"Methapharm, Inc."
14,Jul 15 2022 9:59AM,241957,10,"Nextsource Biotechnology, LLC"
15,Jul 15 2022 9:57AM,241952,12,Hush Hush
31,Jul 15 2022 9:42AM,241950,20,Alumier Labs Inc.
51,Jul 15 2022 9:28AM,241946,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 15 2022 10:50AM,241963,10,Bio-PRF,,,1
1,Jul 15 2022 10:47AM,241947,15,"Carwin Pharmaceutical Associates, LLC",,,1
2,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc",,,2
3,Jul 15 2022 10:36AM,241961,10,Emerginnova,,,1
4,Jul 15 2022 10:30AM,241960,10,Bio-PRF,,,8
5,Jul 15 2022 10:18AM,241958,10,"Methapharm, Inc.",,,1
6,Jul 15 2022 9:59AM,241957,10,"Nextsource Biotechnology, LLC",,1,
7,Jul 15 2022 9:57AM,241952,12,Hush Hush,,,16
8,Jul 15 2022 9:42AM,241950,20,Alumier Labs Inc.,,4,16
9,Jul 15 2022 9:28AM,241946,15,"Virtus Pharmaceuticals, LLC",,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 15 2022 10:50AM,241963,10,Bio-PRF,1,,
1,Jul 15 2022 10:47AM,241947,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc",2,,
3,Jul 15 2022 10:36AM,241961,10,Emerginnova,1,,
4,Jul 15 2022 10:30AM,241960,10,Bio-PRF,8,,
5,Jul 15 2022 10:18AM,241958,10,"Methapharm, Inc.",1,,
6,Jul 15 2022 9:59AM,241957,10,"Nextsource Biotechnology, LLC",,1,
7,Jul 15 2022 9:57AM,241952,12,Hush Hush,16,,
8,Jul 15 2022 9:42AM,241950,20,Alumier Labs Inc.,16,4,
9,Jul 15 2022 9:28AM,241946,15,"Virtus Pharmaceuticals, LLC",7,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 15 2022 10:50AM,241963,10,Bio-PRF,1,,
1,Jul 15 2022 10:47AM,241947,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc",2,,
3,Jul 15 2022 10:36AM,241961,10,Emerginnova,1,,
4,Jul 15 2022 10:30AM,241960,10,Bio-PRF,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 15 2022 10:50AM,241963,10,Bio-PRF,1.0,NaN,NaN
1,Jul 15 2022 10:47AM,241947,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
2,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc",2.0,NaN,NaN
3,Jul 15 2022 10:36AM,241961,10,Emerginnova,1.0,NaN,NaN
4,Jul 15 2022 10:30AM,241960,10,Bio-PRF,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 15 2022 10:50AM,241963,10,Bio-PRF,1.0,0.0,0.0
1,Jul 15 2022 10:47AM,241947,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
2,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc",2.0,0.0,0.0
3,Jul 15 2022 10:36AM,241961,10,Emerginnova,1.0,0.0,0.0
4,Jul 15 2022 10:30AM,241960,10,Bio-PRF,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1935512,30.0,1.0,2.0
12,725819,19.0,1.0,0.0
15,725837,27.0,0.0,0.0
16,1209585,4.0,1.0,0.0
20,725780,90.0,5.0,0.0
21,1209470,3.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1935512,30.0,1.0,2.0
1,12,725819,19.0,1.0,0.0
2,15,725837,27.0,0.0,0.0
3,16,1209585,4.0,1.0,0.0
4,20,725780,90.0,5.0,0.0
5,21,1209470,3.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,30.0,1.0,2.0
1,12,19.0,1.0,0.0
2,15,27.0,0.0,0.0
3,16,4.0,1.0,0.0
4,20,90.0,5.0,0.0
5,21,3.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,12,Released,19.0
2,15,Released,27.0
3,16,Released,4.0
4,20,Released,90.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,1.0,1.0,0.0,1.0,5.0,2.0
Released,30.0,19.0,27.0,4.0,90.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,1.0,0.0,1.0,5.0,2.0
2,Released,30.0,19.0,27.0,4.0,90.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,1.0,0.0,1.0,5.0,2.0
2,Released,30.0,19.0,27.0,4.0,90.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()